## ENVIRONMENT SETTINGS

File Operations

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content
!mkdir dataset
!cp -r /content/drive/MyDrive/analytics_cup ./dataset

Global Variable Definitions

❗️Please do not forget to change the WORKING DIR variable to run the following cells correctly. 

In [ ]:
RANDOM_SEED = 44
WORKING_DIR = '/content/dataset/analytics_cup/'

Install required libraries

In [ ]:
!pip install xgboost

Import Required Python Libraries

In [ ]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from tqdm import tqdm
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

## DATA PREPERATION

Reading the all of the .csv files into Pandas dataframes.

In [ ]:
business_units = pd.read_csv(os.path.join(WORKING_DIR, "business_units.csv"))
customers = pd.read_csv(os.path.join(WORKING_DIR,"customers.csv"))
sales_orders = pd.read_csv(os.path.join(WORKING_DIR,"sales_orders.csv"))
sales_orders_header = pd.read_csv(os.path.join(WORKING_DIR,"sales_orders_header.csv"))
service_map = pd.read_csv(os.path.join(WORKING_DIR,"service_map.csv"))
classification = pd.read_csv(os.path.join(WORKING_DIR,"classification.csv"))

Manipulating obtained dataframes so that we have one final dataframe that includes all of the information provided. 

In [ ]:
# Create a unique tracking id to track each customer
classification['track_id'] = classification.index+1

# Recalculate net value from Sales_Order.csv file since the provided ones are not correct
df_sales_orders_correct_net_values = pd.DataFrame(sales_orders.groupby('Sales_Order')['Net_Value'].sum())
df_sales_orders_correct_net_values = df_sales_orders_correct_net_values.rename(columns={'Net_Value': 'Net_Value_Corrected'})
sales_orders_header = sales_orders_header.merge(df_sales_orders_correct_net_values, on=['Sales_Order'], how='left')

# We do not need the 'Net_Value' column anymore
sales_orders_header.drop('Net_Value', axis=1, inplace= True)

# Join all of the information provided within the .csv files 
anti_join_customers = customers.merge(sales_orders, on=['Sales_Order', 'Item_Position'], how='left', indicator=True)
anti_join_customers = anti_join_customers[anti_join_customers['_merge'] == 'left_only']

anti_join_saleOrders = sales_orders.merge(customers, on=['Sales_Order', 'Item_Position'], how='left', indicator=True)
anti_join_saleOrders = anti_join_saleOrders[anti_join_saleOrders['_merge'] == 'left_only']

index_anti_join_customers = anti_join_customers.index
index_anti_join_saleOrders = anti_join_saleOrders.index

customers.loc[index_anti_join_customers, "Item_Position"] = 0
sales_orders.loc[index_anti_join_saleOrders, "Item_Position"] = 0

# Merge all data in a final dataframe
df = pd.merge(customers, sales_orders,  how='outer', left_on=['Sales_Order','Item_Position'], right_on = ['Sales_Order','Item_Position'])
df = pd.merge(df, classification,  how='outer', left_on=['Customer_ID'], right_on = ['Customer_ID'])
df = pd.merge(df, sales_orders_header,  how='outer', left_on=['Sales_Order'], right_on = ['Sales_Order'])
df = pd.merge(df, service_map ,  how='outer', left_on=['Material_Class'], right_on = ['MATKL_service'])
df = pd.merge(df, business_units ,  how='outer', left_on=['Cost_Center'], right_on = ['Cost_Center'])

In [ ]:
df.drop(['Net_Value','YHKOKRS' ], axis=1, inplace=True)

### Handling the Data

We need to remove Nan values. 

In [ ]:
# Removing rows that have Nan values in the significant columns
df_nan_row_removed = df.dropna(subset=['Sales_Order', 'Customer_ID','Item_Position'])

# Dropping columns that do not contribute to Reseller identification information
df_with_important_columns = df_nan_row_removed.drop('MATKL_service',axis=1)

# Find the columns that have Nan values as an anomaly, 'Reseller','Test_set_id','Release_Date' might include nan values due to the nature of the data
nan_anomaly_cols = [i for i in df_with_important_columns.columns.tolist() if i not in ['Reseller','Test_set_id','Release_Date']]
df_nan_anomaly_removed_cols = df_with_important_columns.dropna(subset=nan_anomaly_cols)

df_anomaly_handled = df_nan_anomaly_removed_cols

We need to handle date/time data to use them as an informative feature for the detection of the Reseller companies. 



In [ ]:
# Identifiying date columns
date_cols = ['Creation_Date','Release_Date']

# Identifiying the indicies that have Release Date col as Nan and replacing those with Creating Date value to not lose data
date_nan_indices = df_anomaly_handled.loc[df_anomaly_handled.Release_Date.isna()].index.values.tolist()
df_anomaly_handled.loc[date_nan_indices, 'Release_Date'] = df_anomaly_handled.loc[date_nan_indices, 'Creation_Date']
df_anomaly_handled['Release_Date'] = pd.to_datetime(df_anomaly_handled['Release_Date'], utc=True)

# Converting object type date&time cols to datetime objects and extracting meaningful information
df_anomaly_handled['week_number_cd'] = pd.to_datetime(df_anomaly_handled['Creation_Date']).dt.week
df_anomaly_handled['month_number_cd'] = pd.to_datetime(df_anomaly_handled['Creation_Date']).dt.month
df_anomaly_handled['year_number_cd'] = pd.to_datetime(df_anomaly_handled['Creation_Date']).dt.year
df_anomaly_handled['season_number_cd'] = pd.to_datetime(df_anomaly_handled['Creation_Date']).dt.quarter

df_anomaly_handled['week_number_rd'] = pd.to_datetime(df_anomaly_handled['Release_Date']).dt.week
df_anomaly_handled['month_number_rd'] = pd.to_datetime(df_anomaly_handled['Release_Date']).dt.month
df_anomaly_handled['year_number_rd'] = pd.to_datetime(df_anomaly_handled['Release_Date']).dt.year
df_anomaly_handled['season_number_rd'] = pd.to_datetime(df_anomaly_handled['Release_Date']).dt.quarter

# Create a new feature named 'time_diff' which shows the diff. between release and creation date. this can be an important attribute. 
df_anomaly_handled['time_diff'] = pd.to_datetime(df_anomaly_handled['Release_Date'],utc=True).dt.floor('d') - pd.to_datetime(df_anomaly_handled['Creation_Date'],utc=True).dt.floor('d')
df_anomaly_handled['time_diff'] = df_anomaly_handled['time_diff'].dt.days

df_anomaly_handled.drop(date_cols, axis=1, inplace=True)

In [ ]:
# Identifiying categorical data columns so that we can encode them to use with ML algorithms
df_encoded = df_anomaly_handled.copy()
categorical_cols = ['Cost_Center', 'Type', 'Customer_ID', 'Material_Code', 'Cost_Center', 'Sales_Organization', 'Creator', 'Document_Type', 'Delivery', 'Business_Unit']

labelEncoder = LabelEncoder()
for col in categorical_cols:
  df_encoded[col] = labelEncoder.fit_transform(df_encoded[col])


In [ ]:
# Create a new feature named 'Suspicious' out of Type column
append_data = []
for i,j in df_encoded.groupby(['Sales_Order','Item_Position']):
  if len(pd.DataFrame(j)['Customer_ID'].unique()) != 1:
    temp = pd.DataFrame(j)[['Sales_Order','Item_Position','Type', 'Customer_ID']]
    temp['Suspicious'] = 1.0
    append_data.append(temp[['Sales_Order','Item_Position', 'Suspicious']])

suspicious_columns = pd.concat(append_data)

df_final = pd.merge(df_encoded, suspicious_columns,  how='outer', left_on=['Sales_Order','Item_Position'], right_on = ['Sales_Order','Item_Position'])
df_final['Suspicious'] = df_final['Suspicious'].fillna(0)

Feature Selection using Correlation

In [ ]:
# Extract correlation matrix and visualize it
corr_matrix = df_final[[c for c in df_final.columns if c not in ['Reseller', 'Test_set_id', 'track_id','Sales_Order', 'Type']]].corr().abs()

#plt.figure(figsize=(24,16))
#sns.heatmap(corr_matrix, annot=True, cmap=plt.cm.Reds)
#plt.show()

def drop_highly_correlated(corr_matrix, threshold, df):
    
    # As correlation matrix is diagonal by its nature we extract upper triangle
    upper_tri_area = np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool)
    upper_tri = corr_matrix.where(upper_tri_area)
    # Choose highly correlated columns according to threshold
    corr_cols = [col for col in upper_tri.columns if any(upper_tri[col] >= threshold)]
    # Drop chosen columns
    df = df.drop(df[corr_cols], axis=1, inplace=False)
    
    return df

df_selected = drop_highly_correlated(corr_matrix, 0.40, df_final[[c for c in df_final.columns if c not in ['Reseller', 'Test_set_id', 'track_id','Sales_Order', 'Type']]])

In [ ]:
# update df_final by dropping not selected columns

preserved_columns = df_selected.columns.tolist()
for c in ['Reseller', 'Test_set_id', 'track_id','Sales_Order', 'Type']:
  preserved_columns.append(c)

df_selected = df_final[preserved_columns]

In [ ]:
# Create a standardized dataset
to_be_transformed_cols = ['Num_Items', 'Net_Value_Corrected', 'time_diff']
sc = StandardScaler()
df_standardized = df_selected.copy()
df_standardized[to_be_transformed_cols] = sc.fit_transform(df_standardized[to_be_transformed_cols])
#df_standardized.describe().T

Split Data into Train, Validation, Test sets

In [ ]:
df_selected_1 = df_selected[df_selected.Reseller == 1]
df_selected_0 = df_selected[df_selected.Reseller == 0].sample(n=len(df_selected_1))
df_selected_final = df_selected_1.append(df_selected_0, ignore_index=False)

In [ ]:
test = df_selected[df_selected.Reseller.isnull()]
train_and_val = df_selected_final[~(df_selected_final.Reseller.isnull())]
train, val = train_test_split(train_and_val, test_size=0.1, random_state=RANDOM_SEED, stratify=train_and_val.Reseller)
# The X,y train splits to train the models
X, y = train.drop(['Reseller', 'Test_set_id', 'track_id','Sales_Order', 'Type', 'Customer_ID'], axis=1), train.Reseller
# The validation set to evaluate the fitted models
X_val, y_val = val.drop(['Reseller', 'Test_set_id', 'track_id','Sales_Order','Type', 'Customer_ID'], axis=1), val.Reseller
# Test set provided as test set
X_test = test.drop(['Reseller', 'Test_set_id', 'track_id','Sales_Order','Type','Customer_ID'], axis=1)

## Model Development

In [ ]:
import statistics
from statistics import multimode
from functools import reduce
from sklearn.metrics import confusion_matrix

# We classify each sales order as if it is a Reseller action or not, then based on the each Customer sales order we perform majority voting 
def mod_to_classify(series):
  result = multimode(series.to_list()) 
  if(len(result) > 1):
    return 0
  else:
    return result[0]

def BAC_metric(y_true, y_pred):
  tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
  # Sensitivity
  sensitivity = tp/(tp+fn)
  # Specificity
  specificity = tn/(tn+fp)  
  bac = (sensitivity + specificity)/(2)
  return bac

In [ ]:
from sklearn.ensemble import RandomForestClassifier
random_forest_clf = RandomForestClassifier(random_state=RANDOM_SEED)
random_forest_clf.fit(X, y)

In [ ]:
import xgboost as xgb
xgb_classifier = xgb.XGBClassifier(objective= 'binary:logistic',
    nthread=4,random_state=RANDOM_SEED)

In [ ]:
clf = xgb_classifier

In [ ]:
# GRID SEARCH
from sklearn.model_selection import GridSearchCV
parameters = {
    'max_depth': range (5, 15, 2),
    'n_estimators': range(60, 200,20),
    'learning_rate': [0.1, 0.01, 0.05],
    'tree_method': ['gpu_hist']
}

grid_search = GridSearchCV(
    estimator=xgb_classifier,
    param_grid=parameters,
    scoring = 'f1',
    n_jobs = 2,
    cv=2,
    verbose=True,
  
)

grid_search.fit(X, y)

grid_search.best_estimator_

In [ ]:
import xgboost as xgb
xgb_classifier = xgb.XGBClassifier(max_depth=16, n_estimators=180, nthread=4, random_state=44)
xgb_classifier.fit(X,y)

In [ ]:
clf = xgb_classifier

In [ ]:
# Validation set evaluation
y_val_pred = clf.predict(X_val)     
val['y_pred'] = y_val_pred
val_pred_results = val.groupby('track_id').agg({'y_pred': [mod_to_classify]})
val_gt_results = val.groupby('track_id').agg({'Reseller': [mod_to_classify]})
val_acc = accuracy_score(val_gt_results, val_pred_results)
val_bac_score = BAC_metric(y_true=val_gt_results, y_pred=val_pred_results)

In [ ]:
val_acc, val_bac_score

# **Test**

In [ ]:
# Test set result 
y_test_pred = clf.predict(X_test)
X_test['track_id'] = test['track_id']
X_test['y_pred'] = y_test_pred 
grouped_results_test_pred = X_test.groupby('track_id').agg({'y_pred': [mod_to_classify]})
prediction = [int(i[0]) for i in grouped_results_test_pred.y_pred.values.tolist()]
sub_data = pd.DataFrame({'prediction': prediction }, index=[int(i) for i in grouped_results_test_pred.index])

In [ ]:
sub_data.to_csv('predictions_HU_Deers_11.csv', index_label='id')